<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Arthropod_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Arthropod Taxonomy Orders Object Detection Dataset](https://www.kaggle.com/datasets/mistag/arthropod-taxonomy-orders-object-detection-dataset/)

In [1]:
import os
import numpy as np
import pandas as pd

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d mistag/arthropod-taxonomy-orders-object-detection-dataset
!unzip *.zip && rm *.zip

In [5]:
images_per_shard = 481
target_width = 1024

CLASSES = ['Lepidoptera', 'Hymenoptera', 'Hemiptera', 'Odonata', 'Diptera', 'Araneae', 'Coleoptera']

RAW_CLASSES = CLASSES + ['_truncated', '_blurred', '_occluded']

In [ ]:
class Progress:

  def __init__(self, maxi, size=100, msg=''):
    self.msg = msg
    self.size = size
    self.maxi = maxi
    self.lock = Lock()
    self.header_printed=False
    self.p = self.__start_progress(maxi)()# () to get the iterator from the generator

  def step(self, reset=False):
    with self.lock:
      if reset:
        self.__init__(self.maxi, self.size, self.msg)
      if not self.header_printed:
        self.__print_header()
      next(self.p)
  
  def __print__header(self):
    print()
    print(f'0%{self.msg: ^self.size-6}100%')
    self.head_printed = True

  def __start_progress(self, maxi):
    def print_progress():
      dx = maxi
      dy = self.size
      d = dy - dx
      for x in range(maxi):
        k = 0
        while d >= 0:
          print('=', end='', flush=True)
          k+=1
          d -= dx
        d += dy
        yield k
    return print_progress

In [ ]:
def no_decorations(ax):
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

def display_detections(images, offsets, resizes, detections, classnames, ground_truth_boxes=[]):
  classes, boxes, scores = [], [], []
  for detection_list in detections:
    for boxlist, ofs, rsz in zip(detection_list, offsets, resizes):
        _, x, y, w, h, score, cls = boxlist
        box = ((x-ofs[1])*rsz, (y-ofs[0])*rsz, w*rsz, h*rsz)
        boxes.append(box)
        classes.append(int(cls))
        scores.append(score)

  display_with_boxes(images, boxes, classes, scores, classnames, ground_truth_boxes)

In [22]:
def display_with_boxes(images, boxes, classes, scores, classnames, ground_truth_boxes=[]):
    N = len(images)
    sqrtN = int(np.ceil(np.sqrt(N)))    
    aspect = sum([im.shape[1]/im.shape[0] for im in images]) / N # mean aspect ratio of images
    fig = plt.figure(figsize=(15, 15/aspect), frameon=False)

    for k in range(N):
      ax = plt.subplot(sqrtN, sqrtN, k+1)
      no_decorations(ax)
      plt.imshow(images[k])

      if ground_truth_boxes:
        for box in ground_truth_boxes[k]:
          x, y, w, h = (box[0], box[1], box[2]-box[0], box[3]-box[1]) #x1 y1 x2 y2 into xywh
          rect = mplr.patches.Rectangle((x, y), w, h, linewidth=4, edgecolor='#FFFFFFA0', facecolor='none')
          ax.add_patch(rect)
      
      for i, (box, cls) in enumerate(zip(boxes[k], classes[k])):
        x,y, w, h = (box[0], box[1], box[2]-box[0], box[3]-box[1]) #x1 y1 x2 y2 into xywh
        label = classnames[cls]
        if scores:
          label += f' {int(scores[k][i])*100}%'
        
        rect = mpl.patches.Rectangle((x, y),w,h,linewidth=4,edgecolor='#00000080',facecolor='none')
        ax.add_patch(rect)
        rect = mpl.patches.Rectangle((x, y),w,h,linewidth=2,edgecolor='#FFFF00FF',facecolor='none')
        ax.add_patch(rect)
        plt.text(x, y, label, size=16, ha="left", va="top", color='#FFFF00FF',
                  bbox=dict(boxstyle="round", ec='#00000080', fc='#0000004E', linewidth=3) )
        plt.text(x, y, label, size=16, ha="left", va="top", color='#FFFF00FF',
                  bbox=dict(boxstyle="round", ec='#FFFF00FF', fc='#0000004E', linewidth=1.5) )
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.02, hspace=0.02)
    plt.show()